In [ ]:
#IMPORT "PACKAGES" (COLLECTIONS OF PRE-DEFINED PYTHON FUNCTIONS)
import pandas as pd
import numpy as np
import math as m
import random as rand
import matplotlib.pyplot as plt
import re
from datetime import datetime

In [ ]:
#READ IN ORIGINAL SHEET (CONVERTED TO .csv IN GOOGLE SHEETS)
df = pd.read_csv("Homebase.csv", header = 1)

In [ ]:
#DEFINING A FUNCTION TO UPDATE COLUMN NAMES LATER
def lower_no_space(word): #I LABELED MY INPUT VARIABLE "WORD" BECAUSE THIS FUNCTION IS FOR WORDS
    
    #REGUGLAR EXPRESSIONS ARE REALLY COMMON FUNCTIONS FOR "STRING" (LETTERS) PARSING
    #THIS ONE REPLACES ONE CHARACTER WITH ANOTHER (SPACES AREN'T GREAT IN CODING)
    word = re.sub(' ', '_', word) #SETTING WORD EQUAL TO THE REGEX OUTPUT TO UPDATE IT
    
    word = re.sub(r'\'', '', word) #\ IS WEIRD IN PYTHON - HAVE TO USE r OTHERWISE THE QUOTES WON'T CLOSE
    
    word = re.sub(r'\(', '', word)
    
    word = re.sub(r'\)', '', word)
    
    word = re.sub('\?', '', word)
    
    word = re.sub('/', '_', word)
    
    #THIS MAKES THE STRING ALL LOWER CASE (I PREFER ALL ONE CASE)
    word = word.lower()
    
    #"RETURN" DEFINES WHAT THE FUNCTION SHOULD OUTPUT
    return word

In [ ]:
#RENAMING COLUMNS
df = df.rename(mapper = lower_no_space, axis = 1) #MAPPER SETS THE FUNCTION TO USE ON EACH COLUMN NAME, 
                                                  #AXIS DEFINES WHICH AXIS (0 OR 1) OF THE DATAFRAME

In [ ]:
#ONE PATIENT ID HAS NO INFO; THIS SYNTAX FILTERS OUT OBSERVATIONS BASED ON A COLUMN VALUE
#HERE, THE COLUMN IS PATIENT_ID; != MEANS "DOES NOT EQUAL"
df = df[df['patient_id'] != '842-2']

In [ ]:
df.head()

In [ ]:
#SEEING IF THERE ARE DUPLICATES; len() GIVES THE LENGTH OF SOMETHING; set() GIVES UNIQUE VALUES IN A LIST
#ANY NON-ZERO VALUE MEANS WE HAVE DUPES IN COLUMN NAMES - NOT GREAT
len(df.columns) - len(set(df.columns))

In [ ]:
#THIS IS CODE I FOUND ONLINE FOR GETTING THE DUPES
s = pd.concat([df.columns.to_series()])

# keep all duplicates only, then extract unique names
res = s[s.duplicated(keep=False)].unique()

res #NOTE THAT ALL HAVE 'treatment' IN IT

In [ ]:
#DATE VARIABLES WERE READ IN AS "OBJECT" TYPES; CONVERTING TO DATETIMEs

#FOR LOOPS APPLY CODE TO EVERY OBJECT IN THE "FOR" STATEMENT
#HERE, THAT MEANS EACH COLUMN (COL) IN THE LIST OF DATAFRAME COLUMNS
for col in df.columns:
    if 'date' in col and 'treatment' not in col:
        df[col] = pd.to_datetime(df[col])
        #DID NOT INCLUDE TREATMENT VARS BECAUSE IT BREAKS THE to_datetime() FUNCTION 

In [ ]:
#NEED TO HAVE SOME VALUE FOR TIME FOR SURVIVAL ANALYSIS, REGARDLESS OF WHETHER OR NOT SOMEONE DIES
#CREATING NEW VAR EQUAL TO LATEST POSSIBLE DATE (PRESUMABLY DEATH DATE, IF DECEASED)
df['max_date'] = df[[x for x in df.columns if 'date' in x and 'treatment' not in x]].max(axis=1)

In [ ]:
#CREATING NEW VARIABLE IN DATAFRAME FOR WEEKS SURVIVED (MAX DATE - DIAGNOSIS DATE)
df['weeks_survived'] = df['max_date'] - df['date_of_initial_diagnosis'] #GET DIFF; RETURNS 'TIMEDELTA OBJECT'
df['weeks_survived'] = df['weeks_survived'].apply(lambda x: abs(x.days)/7) 
                                                 #.apply() W/A LAMBDA APPLIES FUNCTION TO INDIVIDUAL COLUMN VALUES
                                                 #RATHER THAN THE WHOLE COLUMN, WHICH SOMETIMES MESSES UP FUNCTIONS
                                                 #I USUALLY TRY THE SYNTAX IN THE FIRST LINE BEFORE USING LAMBDA
                                                 #BECAUSE THEY ARE INEFFICIENT;
                                                 #THIS LAMBDA FUNCTION GETS DAYS FROM OBJECT, DIVIDES BY 7

In [ ]:
#DEFINING 'CENSORED' FOR SURVIVAL ANALYSIS; 
#CENSORED MEANS WE DON'T KNOW HOW MANY WEEKS THEY'LL SURVIVE OR THAT THEY DIED UNRELATED TO THE DISEASE
#THE POINT IS THAT WE DON'T KNOW HOW LONG THEY WOULD HAVE SURVIVED THE DISEASE
df['censored'] = df['death_related_to_disease'].apply(lambda x: 0 if x == 'Yes' else 1)


In [ ]:
#CREATING EMPTY DATA DICTIONARY
#BASIC DICTIONARY STRUCTURE IS dict[key] = value
corr_dict = {}

#DEFINING NEW DATAFRAME; .corr() RETURNS CORRELATION COEFFICIENTS BETWEEN COLUMNS IN ORIGINAL DF
c = df.corr()

#FILLING DATA DICTIONARY WITH COLUMN NAME AND A LIST WITH IT'S CORRELATION COEFF WITH WEEKS SURVIVED
#AS WELL AS OBSERVATIONS IN THE DF (NON-NULLS); I USE THESE BOTH TO DETERMINE VARS OF INTEREST LATER
for other in c.columns:
    corr_dict[other] = [c['weeks_survived'][other], len(df) - df[other].isna().sum()]


In [ ]:
#FILTERING OUT:
#1) SMALL (ARBITRARILY DEFINED) COEFFICIENTS 
#2) COLUMNS THAT WERE THE SURVEY QUESTIONS
#3) COLUMNS WITH MORE THAN HALF NULLS (ARBITRARY CUTOFF)
corr_dict = dict(filter(lambda elem: abs(elem[1][0]) >= .1 and \
                        '?' not in elem[0] and \
                        elem[1][1]/len(df) >= 0.5, corr_dict.items()))

In [ ]:
#CONVERTING DICTIONARY TO DF TO MAKE SAVING A CSV EASIER
df_corr = pd.DataFrame(corr_dict.items(), columns = ['column', 'correlation_obs'])

In [ ]:
#SAVING NEW CSVs TO USE IN R DATA VIZ
df.to_csv('Homebase_updated.csv')
df_corr.to_csv('notable_correlations.csv')